In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import json

In [9]:

path_02 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/*.png"
path_03 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/*.png"
images_02 = sorted(glob.glob(path_02))
images_03 = sorted(glob.glob(path_03))

# for at finde flere chessboards på et billede, laver vi bounding boxes omkring hvert chessboard således at 
# hver roi kun har ét chessboard i sig, som vi så kan køre findChessboardCornersSB på. 
with open("rois_02.json", "r") as f:
    rois_02 = json.load(f)[:13]

with open("rois_03.json", "r") as f:
    rois_03 = json.load(f)[:13]

# chessboard mønstre 
patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# hvert roi indeholder et chessboard med et af de fire mønstre 
roi_patterns = [0,1,2,3,0,1,2,3,0,1,2,3,0]  

# Subpixel refinement criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objpoints = []
imgpoints_02 = []
imgpoints_03 = []

# looper over alle 19 par med 1 billede fra 02, og 1 billede fra 03
for f02, f03 in zip(images_02, images_03):
    gray02 = cv2.cvtColor(cv2.imread(f02), cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(cv2.imread(f03), cv2.COLOR_BGR2GRAY)

    corners02_img = []
    corners03_img = []
    objpoints_img = []

    # Looper over rois
    for idx, (roi02, roi03) in enumerate(zip(rois_02, rois_03)):
        pat = patterns[roi_patterns[idx]]
        ps = pat["pattern_size"]
        sq = pat["square_size"]

        # Cropper rois 
        x1,y1,x2,y2 = roi02
        x1b,y1b,x2b,y2b = roi03
        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        ok02, c02 = cv2.findChessboardCornersSB(crop02, ps) #finder chessboard i hvert roi 
        ok03, c03 = cv2.findChessboardCornersSB(crop03, ps)

        if not (ok02 and ok03):
            continue  # skip if board not found in both cameras, det er vigtigt vi finder det samme board i begge billeder i parret 

        # corner refining tingting 
        c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria)
        c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria)

        c02_full = c02 + np.array([[x1, y1]], dtype=np.float32)
        c03_full = c03 + np.array([[x1b, y1b]], dtype=np.float32)

        objp = np.zeros((ps[0]*ps[1],3), np.float32) #object point til camera calibration for hvert board 
        objp[:,:2] = np.mgrid[0:ps[0], 0:ps[1]].T.reshape(-1,2)
        objp *= sq

        corners02_img.append(c02_full.reshape(-1,2))
        corners03_img.append(c03_full.reshape(-1,2))
        objpoints_img.append(objp)

    if len(corners02_img) > 0:
        imgpoints_02.append(np.vstack(corners02_img)) # vi bruger kun billeder hvor mindst et skakbræt blev fundet i begge par 
        imgpoints_03.append(np.vstack(corners03_img))
        objpoints.append(np.vstack(objpoints_img))

print(f"Collected {len(objpoints)} valid image pairs for calibration.")

# Camera calibration for kamera 02 og 03 
ret02, mtx02, dist02, rvecs02, tvecs02 = cv2.calibrateCamera(
    objpoints, imgpoints_02, gray02.shape[::-1], None, None
)
ret03, mtx03, dist03, rvecs03, tvecs03 = cv2.calibrateCamera(
    objpoints, imgpoints_03, gray03.shape[::-1], None, None
)

# image size og criteria til stereo calibration
image_size = (gray02.shape[1], gray02.shape[0])
criteria_stereo = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)

# Stereo calibration
ret_stereo, mtx02, dist02, mtx03, dist03, R, T, E, F = cv2.stereoCalibrate(
    objpoints,
    imgpoints_02,
    imgpoints_03,
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    criteria=criteria_stereo,
    flags=cv2.CALIB_FIX_INTRINSIC
)

print("Rotation:\n", R)
print("Translation:\n", T)


Collected 19 valid image pairs for calibration.
Rotation:
 [[ 0.99760188 -0.04461287 -0.05291683]
 [ 0.06111785  0.92661966  0.37099948]
 [ 0.03248242 -0.37334394  0.92712415]]
Translation:
 [[ 0.00630834]
 [-0.01502299]
 [-0.00511162]]


In [10]:
# stereo rectification 
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    R, T,
    flags=cv2.CALIB_ZERO_DISPARITY,
    alpha=1  # 0 = zoomer, 1 = fuldt billede
)

print("Rectification matrix R1:\n", R1)
print("Rectification matrix R2:\n", R2)
print("Projection matrix P1:\n", P1)
print("Projection matrix P2:\n", P2)
print("Disparity-to-depth matrix Q:\n", Q)

# Compute undistort/rectify maps (for later use on any stereo pair)
left_map_x, left_map_y = cv2.initUndistortRectifyMap(
    mtx02, dist02, R1, P1, image_size, cv2.CV_32FC1
)
right_map_x, right_map_y = cv2.initUndistortRectifyMap(
    mtx03, dist03, R2, P2, image_size, cv2.CV_32FC1
)



Rectification matrix R1:
 [[ 0.94802818  0.29177469  0.1269256 ]
 [-0.30503537  0.71990689  0.62344807]
 [ 0.09053176 -0.62976313  0.77149361]]
Rectification matrix R2:
 [[ 0.92602128  0.37539493  0.03953772]
 [-0.36941186  0.87973568  0.29933261]
 [ 0.0775852  -0.29179407  0.9533293 ]]
Projection matrix P1:
 [[-1.49236653e+02  0.00000000e+00  1.23584194e+03  0.00000000e+00]
 [ 0.00000000e+00 -1.49236653e+02  1.40077285e+03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Projection matrix P2:
 [[-1.49236653e+02  0.00000000e+00  1.23584194e+03  0.00000000e+00]
 [ 0.00000000e+00 -1.49236653e+02  1.40077285e+03  2.54847152e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Disparity-to-depth matrix Q:
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -1.23584194e+03]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.40077285e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.49236653e+02]
 [ 0.00000000e+00  0.00000000e+

In [ ]:
#formatterer resultaterne så vi kan sammenligne med deres rect 
# formatterer til pretty print matrice/vektor 

def print_matrix(name, M): 
    M_flat = M.reshape(-1)
    print(f"{name}: " + " ".join(f"{v:.6e}" for v in M_flat))

def print_vector(name, v):
    v_flat = v.reshape(-1)
    print(f"{name}: " + " ".join(f"{x:.6e}" for x in v_flat))

print("\nKamera 02 resultater")

#kamera 02 
print("S_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_02", mtx02)
print_vector("D_02", dist02)

# stereo R,T ransformen fra 02 til 03
print_matrix("R_02", np.eye(3))
print_vector("T_02", np.zeros((3,1)))

print("\nRectified resultater for 02")
print("S_rect_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_02", R1)
print_matrix("P_rect_02", P1)


print("\nKamera 03 resultater")

print("S_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_03", mtx03)
print_vector("D_03", dist03)

# kamera 03 pose ift kamera 02, fra stereoCalibrate
print_matrix("R_03", R)
print_vector("T_03", T)

print("\nRectified resultater for 03")
print("S_rect_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_03", R2)
print_matrix("P_rect_03", P2)

print("\n Q matrice:")
print_matrix("Q", Q)


Kamera 02 resultater
S_02: 1.392000e+03 5.120000e+02
K_02: 4.978166e+02 0.000000e+00 6.955000e+02 0.000000e+00 7.962095e+01 2.555000e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_02: -1.079375e-02 1.578055e-05 -1.770847e-03 1.603822e-04 -6.126344e-09
R_02: 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00
T_02: 0.000000e+00 0.000000e+00 0.000000e+00

Rectified resultater for 02
S_rect_02: 1.392000e+03 5.120000e+02
R_rect_02: 9.480282e-01 2.917747e-01 1.269256e-01 -3.050354e-01 7.199069e-01 6.234481e-01 9.053176e-02 -6.297631e-01 7.714936e-01
P_rect_02: -1.492367e+02 0.000000e+00 1.235842e+03 0.000000e+00 0.000000e+00 -1.492367e+02 1.400773e+03 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00

Kamera 03 resultater
S_03: 1.392000e+03 5.120000e+02
K_03: 4.207215e+02 0.000000e+00 6.954999e+02 0.000000e+00 4.451622e+01 2.555002e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_03: -2.464261e-02 8.479832e-06 1.5417